In [6]:
# from functools import partial
from pathlib import Path
# from typing import Optional, Tuple
# import cv2
# import fire
import numpy as np
import torch
import torch.nn.functional as F
from accelerate import Accelerator
from PIL import Image
from scipy.sparse.linalg import eigsh
# from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.decomposition import PCA
# from torchvision.utils import draw_bounding_boxes
from tqdm import tqdm
import extract_utils as utils
from torch.utils.data import Dataset, DataLoader
from lightly.loss import NegativeCosineSimilarity
from lightly.models.modules import SimSiamPredictionHead, SimSiamProjectionHead
from torch import nn
import torchvision
from scipy.ndimage import affine_transform
from matplotlib import pyplot as plt

## Extract Eigen Vectors

In [7]:
images_root="/home/phdcs2/Hard_Disk/Datasets/Deep-Spectral-Segmentation/data/object-segmentation/ECSSD/images"
features_dir="/home/phdcs2/Hard_Disk/Datasets/Deep-Spectral-Segmentation/data/object-segmentation/ECSSD/features/dino_vits16"
output_dir="/home/phdcs2/Hard_Disk/Datasets/Deep-Spectral-Segmentation/data/object-segmentation/ECSSD/eigs_dot1PCA64pred_dssubmax_10_histintersection"
which_matrix= 'laplacian'
which_color_matrix= 'knn'
which_features= 'k'
normalize=True
threshold_at_zero=True
lapnorm= True
K= 5
image_downsample_factor = None
image_color_lambda = 0.0
multiprocessing = 0
batch_size=2
epochs=10

## Incorporating SimSiam with PCA projector and a predictor

In [8]:
def histogram_intersection(hist1, hist2):
    """
    Compute the histogram intersection between two histograms.
    Args:
        hist1 (torch.Tensor): Histogram 1.
        hist2 (torch.Tensor): Histogram 2.
    Returns:
        torch.Tensor: Histogram intersection.
    """
    minima = torch.min(hist1, hist2)
    intersection = torch.sum(minima)
    return intersection

In [9]:
# inputs = list(enumerate(sorted(Path(features_dir).iterdir())))
# for inp in tqdm(inputs[:1]):
#     index, features_file = inp
#     print(index, features_file)
#      # Load
#     data_dict = torch.load(features_file, map_location='cpu')
#     # print(data_dict.keys())   #['k', 'indices', 'file', 'id', 'model_name', 'patch_size', 'shape']
#     # print("shape=", data_dict['shape'], "k shape", data_dict['k'].shape, "patch_size=", data_dict['patch_size'])
#     image_id = data_dict['file'][:-4]
#     print(image_id)
#     # Load
#     output_file = str(Path(output_dir) / f'{image_id}.pth')
#     if Path(output_file).is_file():
#         print(f'Skipping existing file {str(output_file)}')
#         # break
#         # return  # skip because already generated
#
#     # Load affinity matrix
#     feats = data_dict[which_features].squeeze().cuda()
#     # print(feats.shape)
#     tanh_feats=torch.tanh(feats)
#     print("tanh_feats.shape=", tanh_feats.shape)
#     rows=tanh_feats.shape[0]
#     W_feat_jsd=torch.empty(rows, rows)
#     for i in range(rows):
#         t1=tanh_feats[i]
#         # print("t1.shape=", t1.shape)
#         min_t1=torch.min(t1)
#         max_t1=torch.max(t1)
#         # print("minimum of t1=", min_t1)
#         # print("maximum of t1=", max_t1)
#         # Calculate the histogram
#         hist1 = torch.histc(t1, bins=20, min=-1, max=1)
#         # print(hist1)
#         for j in range(rows):
#             t2=tanh_feats[i]
#             # t1_float = t1.float()
#             # print("t1.shape=", t1.shape)
#             min_t1=torch.min(t2)
#             max_t1=torch.max(t2)
#             # print("minimum of t1=", min_t1)
#             # print("maximum of t1=", max_t1)
#             # print()
#             # Calculate the histogram
#             hist2 = torch.histc(t1, bins=20, min=-1, max=1)
#             # print(hist2)
#             jsd=js_divergence(hist1, hist2)
#             W_feat_jsd[i,j]=jsd.item()
#
#
#
#     # hist = torch.histc(tanh_feats.cpu(), bins=20, min=-100, max=100)
#     # x=np.linspace(-100, 100, 20)

In [10]:
#Incorporating JS-Divergence in addition to Cosine Affinity
utils.make_output_dir(output_dir)
inputs = list(enumerate(sorted(Path(features_dir).iterdir())))
device = "cuda" if torch.cuda.is_available() else "cpu"
for inp in tqdm(inputs):
    index, features_file = inp
    print(index, features_file)
     # Load
    data_dict = torch.load(features_file, map_location='cpu')
    # print(data_dict.keys())   #['k', 'indices', 'file', 'id', 'model_name', 'patch_size', 'shape']
    # print("shape=", data_dict['shape'], "k shape", data_dict['k'].shape, "patch_size=", data_dict['patch_size'])
    image_id = data_dict['file'][:-4]
    print(image_id)
    # Load
    output_file = str(Path(output_dir) / f'{image_id}.pth')
    if Path(output_file).is_file():
        print(f'Skipping existing file {str(output_file)}')
        # break
        # return  # skip because already generated

    # Load affinity matrix
    feats = data_dict[which_features].squeeze().cuda()
    # print("Without normalizing, Features Shape is",feats.shape)
    if normalize:
        feats = F.normalize(feats, p=2, dim=-1)
    # print("After normalization, Features Shape",feats.shape)
    # print("which_matrix=", which_matrix)
    # Eigenvectors of affinity matrix
    if which_matrix == 'affinity_torch':
        W = feats @ feats.T
        # W_feat=contrastive_affinity(feats, feats.T)
        # print("W shape=", W.shape)
        if threshold_at_zero:
            W = (W * (W > 0))
            # print("W shape=", W.shape)
        eigenvalues, eigenvectors = torch.eig(W, eigenvectors=True)
        eigenvalues = eigenvalues.cpu()
        eigenvectors = eigenvectors.cpu()
        print("which matrix=",which_matrix, "eigenvalues shape", eigenvalues.shape, "eigenvectors shape", eigenvectors.shape)


    # Eigenvectors of affinity matrix with scipy
    elif which_matrix == 'affinity_svd':
        USV = torch.linalg.svd(feats, full_matrices=False)
        eigenvectors = USV[0][:, :K].T.to('cpu', non_blocking=True)
        eigenvalues = USV[1][:K].to('cpu', non_blocking=True)
        print("which matrix=",which_matrix,"eigenvalues shape", eigenvalues.shape, "eigenvectors shape", eigenvectors.shape)

    # Eigenvectors of affinity matrix with scipy
    elif which_matrix == 'affinity':
        # print("Without normalizing, Features Shape is",feats.shape)
        W = (feats @ feats.T)
        # W_feat=contrastive_affinity(feats, feats.T)
        # print("W shape=", W.shape)
        if threshold_at_zero:
            W = (W * (W > 0))
        W = W.cpu().numpy()
        # print("W shape=", W.shape)
        eigenvalues, eigenvectors = eigsh(W, which='LM', k=K)
        eigenvectors = torch.flip(torch.from_numpy(eigenvectors), dims=(-1,)).T
        print("which matrix=",which_matrix, "eigenvalues shape", eigenvalues.shape, "eigenvectors shape", eigenvectors.shape)

    # Eigenvectors of matting laplacian matrix
    elif which_matrix in ['matting_laplacian', 'laplacian']:

        # Get sizes
        B, C, H, W, P, H_patch, W_patch, H_pad, W_pad = utils.get_image_sizes(data_dict)
        if image_downsample_factor is None:
            image_downsample_factor = P
        H_pad_lr, W_pad_lr = H_pad // image_downsample_factor, W_pad // image_downsample_factor

        # Upscale features to match the resolution
        if (H_patch, W_patch) != (H_pad_lr, W_pad_lr):
            feats = F.interpolate(
                feats.T.reshape(1, -1, H_patch, W_patch),
                size=(H_pad_lr, W_pad_lr), mode='bilinear', align_corners=False
            ).reshape(-1, H_pad_lr * W_pad_lr).T

        ### Feature affinities
        # print("Without normalizing, Features Shape is",feats.shape)

        W_feat_ds = (feats @ feats.T)
        tanh_feats=torch.tanh(feats)
        print("tanh_feats.shape=", tanh_feats.shape)
        rows=tanh_feats.shape[0]
        W_feat_hist_intersect=torch.zeros(rows, rows)
        for i in range(rows):
            t1=tanh_feats[i]
            # print("t1.shape=", t1.shape)
            min_t1=torch.min(t1)
            max_t1=torch.max(t1)
            # print("minimum of t1=", min_t1)
            # print("maximum of t1=", max_t1)
            # Calculate the histogram
            hist1 = torch.histc(t1, bins=20, min=-1, max=1)
            # print(hist1)
            for j in range(rows):
                t2=tanh_feats[i]
                # t1_float = t1.float()
                # print("t1.shape=", t1.shape)
                min_t1=torch.min(t2)
                max_t1=torch.max(t2)
                # print("minimum of t1=", min_t1)
                # print("maximum of t1=", max_t1)
                # print()
                # Calculate the histogram
                hist2 = torch.histc(t1, bins=20, min=-1, max=1)
                # print(hist2)
                hist_intersect=histogram_intersection(hist1, hist2)
                W_feat_hist_intersect[i,j]=hist_intersect.item()
        W_feat_ds=W_feat_ds.to(device)
        W_feat_hist_intersect=W_feat_hist_intersect.to(device)
        print("W_feat_ds.shape=", W_feat_ds.shape)
        print("W_feat_jsd.shape=", W_feat_hist_intersect.shape)
        W_feat=W_feat_ds+W_feat_hist_intersect
        print(type(W_feat_hist_intersect))
        # print(W_feat.shape)
        # print(W_feat)
        if threshold_at_zero:
            W_feat = (W_feat * (W_feat > 0))
        W_feat = W_feat / W_feat.max()  # NOTE: If features are normalized, this naturally does nothing
        # W_feat = W_feat.cpu().numpy()
        W_feat = W_feat.detach().cpu().numpy()
        # print("W_feat shape=",W_feat.shape)

        ### Color affinities
        # If we are fusing with color affinites, then load the image and compute
        if image_color_lambda > 0:

            # Load image
            image_file = str(Path(images_root) / f'{image_id}.jpg')
            image_lr = Image.open(image_file).resize((W_pad_lr, H_pad_lr), Image.BILINEAR)
            image_lr = np.array(image_lr) / 255.

            # Color affinities (of type scipy.sparse.csr_matrix)
            if which_color_matrix == 'knn':
                W_lr = utils.knn_affinity(image_lr / 255)
            elif which_color_matrix == 'rw':
                W_lr = utils.rw_affinity(image_lr / 255)

            # Convert to dense numpy array
            W_color = np.array(W_lr.todense().astype(np.float32))
            # print("W_color shape", W_color.shape)

        else:

            # No color affinity
            W_color = 0

        # Combine
        W_comb = W_feat + W_color * image_color_lambda  # combination
        D_comb = np.array(utils.get_diagonal(W_comb_tmp).todense())  # is dense or sparse faster? not sure, should check
        # W_comb=np.nan_to_num(W_comb_tmp, nan=0.0)
        # D_comb=np.nan_to_num(D_comb_tmp, nan=0.0)
        # print("W_comb shape= ", W_comb.shape, "D_comb shape",  D_comb.shape)
        if lapnorm:
            try:
                eigenvalues, eigenvectors = eigsh(D_comb - W_comb, k=K, sigma=0, which='LM', M=D_comb)
            except:
                eigenvalues, eigenvectors = eigsh(D_comb - W_comb, k=K, which='SM', M=D_comb)
        else:
            try:
                eigenvalues, eigenvectors = eigsh(D_comb - W_comb, k=K, sigma=0, which='LM')
            except:
                eigenvalues, eigenvectors = eigsh(D_comb - W_comb, k=K, which='SM')
        eigenvalues, eigenvectors = torch.from_numpy(eigenvalues), torch.from_numpy(eigenvectors.T).float()
    print("eigenvalues shape", eigenvalues.shape, "eigenvectors shape", eigenvectors.shape)
    # Sign ambiguity
    for k in range(eigenvectors.shape[0]):
        if 0.5 < torch.mean((eigenvectors[k] > 0).float()).item() < 1.0:  # reverse segment
            eigenvectors[k] = 0 - eigenvectors[k]

    # Save dict
    output_dict = {'eigenvalues': eigenvalues, 'eigenvectors': eigenvectors}
    torch.save(output_dict, output_file)

  0%|          | 0/1000 [00:00<?, ?it/s]

0 /home/phdcs2/Hard_Disk/Datasets/Deep-Spectral-Segmentation/data/object-segmentation/ECSSD/features/dino_vits16/0001.pth
0001
tanh_feats.shape= torch.Size([400, 384])
W_feat_ds.shape= torch.Size([400, 400])
W_feat_jsd.shape= torch.Size([400, 400])
<class 'torch.Tensor'>


  0%|          | 1/1000 [01:18<21:48:36, 78.60s/it]

eigenvalues shape torch.Size([5]) eigenvectors shape torch.Size([5, 400])
1 /home/phdcs2/Hard_Disk/Datasets/Deep-Spectral-Segmentation/data/object-segmentation/ECSSD/features/dino_vits16/0002.pth
0002
tanh_feats.shape= torch.Size([400, 384])


  0%|          | 2/1000 [02:43<22:47:54, 82.24s/it]

...

eigenvalues shape torch.Size([5]) eigenvectors shape torch.Size([5, 400])
998 /home/phdcs2/Hard_Disk/Datasets/Deep-Spectral-Segmentation/data/object-segmentation/ECSSD/features/dino_vits16/0999.pth
0999
tanh_feats.shape= torch.Size([375, 384])
W_feat_ds.shape= torch.Size([375, 375])
W_feat_jsd.shape= torch.Size([375, 375])
<class 'torch.Tensor'>


100%|█████████▉| 999/1000 [27:02:11<01:29, 89.61s/it]

eigenvalues shape torch.Size([5]) eigenvectors shape torch.Size([5, 400])
999 /home/phdcs2/Hard_Disk/Datasets/Deep-Spectral-Segmentation/data/object-segmentation/ECSSD/features/dino_vits16/1000.pth
1000
tanh_feats.shape= torch.Size([400, 384])
W_feat_ds.shape= torch.Size([400, 400])
W_feat_jsd.shape= torch.Size([400, 400])
<class 'torch.Tensor'>


100%|██████████| 1000/1000 [27:03:31<00:00, 97.41s/it]

eigenvalues shape torch.Size([5]) eigenvectors shape torch.Size([5, 400])
